In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import to_categorical

# Load the training and testing datasets from files
train = pd.read_csv("OneHot_trainingSet_Names.csv")
test = pd.read_csv("OneHot_testingSet_Names.csv")

# Extract the input features and output labels from the datasets
xtrain = train.iloc[:, 1:].values  # Input features in training set
ytrain = train["piece"].values     # Output labels in training set
xtest = test.iloc[:, 1:].values    # Input features in testing set
ytest = test["piece"].values       # Output labels in testing set

# One-hot encode the output labels
ytrain = to_categorical(ytrain)
ytest = to_categorical(ytest)

# Reshape the input data to match the LSTM input shape
xtrain = xtrain.reshape(xtrain.shape[0], xtrain.shape[1], 1)
xtest = xtest.reshape(xtest.shape[0], xtest.shape[1], 1)

# Define the input and output dimensions based on the reshaped input data and output data
in_dim = (xtrain.shape[1], xtrain.shape[2])
out_dim = ytrain.shape[1]

# Create and compile the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=in_dim, activation="relu"))
model.add(Dense(out_dim, activation='softmax', kernel_regularizer='l2'))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

# Train the model
model.fit(xtrain, ytrain, epochs=100, batch_size=12, verbose=1)

# Evaluate the model on the testing data
ypred = model.predict(xtest)
loss, accuracy = model.evaluate(xtest, ytest)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Get the predicted composer label with the highest probability
predicted_composer = np.argmax(ypred, axis=1)

# Determine the number of classes (composers) in the output
num_classes = ytrain.shape[1]

# Generate the composer labels array
composer_labels = np.arange(num_classes)

# Convert the predicted label back to its original representation
predicted_composer = composer_labels[predicted_composer]

# Get the piece names from the testing set
piece_names = test["piece"].values

# Get the top 3 predicted composers and their probabilities
top_composers = np.argsort(ypred, axis=1)[:, -3:]
top_probabilities = np.sort(ypred, axis=1)[:, -3:]

# Convert the predicted labels back to their original representations
predicted_composers = composer_labels[top_composers]

# Print the predicted composers and their probabilities
count = 20
for composers, piece_name, probabilities in zip(predicted_composers, piece_names, top_probabilities):
    if count % 20 == 0:
        print("Predicted Composers:", composers[::-1])
        print("Probabilities:", probabilities[::-1])
        print("Piece Name:", piece_name)
        count += 1
    else:
        count += 1
